In [ ]:
from  datasets  import  load_dataset, DatasetDict
from transformers import AutoTokenizer
from transformers import GPT2LMHeadModel, AutoConfig, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments





# Downloading and loading a dataset from the hub.
raw_datasets = load_dataset("ammarnasr/Customizable-Code-Assistant-Data")

# Filtering the dataset to only Python examples.
raw_datasets = raw_datasets.filter(lambda example: example['language'] == 'Python')

# Splitting the dataset into train, test, and validation sets.
train_testvalid = raw_datasets['train'].train_test_split(0.1)
test_valid = train_testvalid['test'].train_test_split(0.5)
raw_datasets = DatasetDict({
    'train': train_testvalid['train'],
    'test': test_valid['test'],
    'valid': test_valid['train']})


# Tokenizing the dataset
tokenizer = AutoTokenizer.from_pretrained("huggingface-course/code-search-net-tokenizer")
context_length = 128
def tokenize(element):
    outputs = tokenizer(
        element["text"],
        truncation=True,
        max_length=context_length,
        return_overflowing_tokens=True,
        return_length=True,
    )
    input_batch = []
    for length, input_ids in zip(outputs["length"], outputs["input_ids"]):
        if length == context_length:
            input_batch.append(input_ids)
    return {"input_ids": input_batch}
tokenized_datasets = raw_datasets.map(tokenize, batched=True, remove_columns=raw_datasets["train"].column_names)
print(f"Tokenized dataset: {tokenized_datasets['train'].num_rows} training samples")
      
# Preparing the model
config = AutoConfig.from_pretrained(
    "gpt2",
    vocab_size=len(tokenizer),
    n_ctx=context_length,
    bos_token_id=tokenizer.bos_token_id,
    eos_token_id=tokenizer.eos_token_id,
)
model = GPT2LMHeadModel(config)
model_size = sum(t.numel() for t in model.parameters())
print(f"GPT-2 size: {model_size/1000**2:.1f}M parameters")

# Preparing the data collator
tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

# Preparing the training arguments
args = TrainingArguments(
    output_dir="gpt2-from-scratch-customizable-code-assistant",
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    evaluation_strategy="steps",
    eval_steps=5_000,
    logging_steps=5_000,
    num_train_epochs=1,
    weight_decay=0.1,
    warmup_steps=1_000,
    lr_scheduler_type="cosine",
    learning_rate=5e-4,
    save_steps=5_000,
)
trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    args=args,
    data_collator=data_collator,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["valid"],
)


In [ ]:
# Transformer Math : 
# basic math related to computation and memory usage for transformers

# 1. Information About Local Accelerator (GPU/CPU)
# This includes all the available devices and each one's :
# Name, Compute Capability, Multiprocessors, CUDA Cores, Concurrent threads, GPU clock, Memory clock, Total Memory and Free Memory.
from cuda_utils import  main, gpu_flops
import json
cuda_info = main()
print(json.dumps(cuda_info,  indent=4))

# 2. Compute Requirements
# The basic equation giving the cost to train a transformer model is given by:
#                                    C = tao*T = 6*P*D
# where:
# C is the compute required to train the transformer model, in total floating point operations (FLOPs)
# C = C_forward + C_backward
# C_forward  =  2*P*D
# C_backward =  4*P*D
# tao is the aggregate throughput of your hardware setup (tao = (No. of GPUs) * (Actual FLOPs/GPU), in FLOPs
# T is the time spent training the model, in seconds
# P is the number of parameters in the transformer model
# D is the dataset size, in tokens

#3. Estimating GPU Actual FLOPs (tao)
# Estimating GPU FLOPs accurately can be challenging due to the complexity of GPU architectures and optimizations. However, here is a simple formula to estimate a GPU FLOP/s:
# Total GPU FLOPS/s = GPU clock * cores * flops_per_clock_cycle * fp_precision
for gpu_no in cuda_info:
    gpu = cuda_info[gpu_no]
    print(f"GPU: {gpu['Name']}")
    print(f"GPU FLOPS: {gpu_flops(gpu):.2f} TFLOPS")
    tao = gpu_flops(gpu) # TFLOPS

#4. Estimating Model Parameters (P)
P = model.num_parameters()
print(f"Model Parameters: {P/1000**2:.2f}M")

#5. Estimating Dataset Size (D)
D = 0
for input_ids in tokenized_datasets["train"]["input_ids"]:
    D += len(input_ids)
print(f"Dataset Size: {D} tokens")

#6. Estimating Training Time (T)
# Calculate training time based on the formula tao*T = 6*P*D. Note that ao needs to be in FLOPS/s not TFLOPS/s, to convert TFLOPS/s to FLOPS/s multiply by 1000**3.
T = 6*P*D/(tao*1000**3)
print(f"Training Time: {T/3600:.2f} hours")


#7. Parameter vs Dataset Tradeoffs
# Although strictly speaking you can train a transformer for as many tokens as you like, the number of tokens trained can highly impact both the computing costs and the final model performance making striking the right balance important.
# compute optimal language model has a number of parameters and a dataset size that satisfies the approximation D = 20*P.
optimal_D = 20*P
print(f"Optimal Dataset Size: {optimal_D / 1000**2:.2f}M tokens ({optimal_D / D:.2f}x current dataset size)")





In [ ]:
from cuda_utils import  main, list_common_gpus, custome_gpu_info
import json

available_gpus = list_common_gpus()
print(f"Available GPUs: {available_gpus}")


t4_cuda_info = custome_gpu_info('NVIDIA T4')
cuda_info = main(verbose=False)
print(json.dumps(t4_cuda_info,  indent=4))
print('------------------')
print(json.dumps(cuda_info,  indent=4))



In [87]:
import pandas as pd
df = pd.read_csv('llms_info.csv')



In [88]:
df

,Date,Model,Arch.,Size,Vocab,Context,Init. from,Dataset,Training,PL
0,2019-12,CuBERT,BERT,350M,50K,1024,-,9.3B,93B,1
1,2020-02,CodeBERT,RoBERTa,125M,50K,512,RoBERTa,20GB,105B,6
2,2020-09,GraphCode-BERT,RoBERTa,125M,50K,640,CodeBERT,20GB,131B,6
3,2021-08,SynCoBERT,RoBERTa,125M,50K,512,CodeBERT,20GB,7B,6
4,2021-10,DISCO,BERT,100M,20K,512,-,1.8GB,-,2
5,2022-05,Code-MVP,RoBERTa,125M,50K,512,GraphCodeBERT,2GB,39B,1
6,2020-05,GPT-C,GPT-2,374M,60K,1024,-,11B,270B,4
7,2021-02,CodeGPT,GPT-2,124M,50K,1024,GPT-2,2GB,-,1
8,2022-02,PolyCoder,GPT-2,160M-2.7B,50K,2048,-,254GB,39B,12
9,2022-03,CodeGen-Multi(Mono),GPT-3,350M-16.1B,50K,2048,-,1.6TB(1.8TB)/506B(577B),1T(1.2T),6(1)


In [1]:
import csv

datasets = [
    ["Dataset", "Language", "Size", "Description"],
    ["CodeXGLUE", "Python", "10k samples", "Benchmark dataset for code intelligence"],
    ["HumanEval", "Python", "77k samples", "Diverse code samples for training foundations models"], 
    ["CodeParrot", "Python", "35 million samples", "Large dataset of Python functions for few-shot learning"],
    ["CodeForce", "C++", "435k problems", "Competitive programming challenges"],
    ["GitHub", "Various", "Billions of lines", "Open source code from public GitHub repositories"],
]

with open('code_datasets.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(datasets)

In [14]:
range_start_X = 10**6
range_end_X = 10**10
range_step_X = 5*10**6

x_vals = list(range(range_start_X, range_end_X, range_step_X))


In [15]:
len(x_vals)

2000